In [1]:
# Import librairies to work with data
import pandas as pd
import numpy as np
import re
# Geocoding our Chef-lieux
from geopy.geocoders import Nominatim

# Data Cleaning - Exploration

In [2]:
# Creating a mapping dictionnary to get town names and departments
com_df = pd.read_excel('table-appartenance-geo-communes-18_V2.xls', sheet_name=0, skiprows=5, usecols=['CODGEO',
                                                                                                       'LIBGEO',
                                                                                                       'DEP'],
                       dtype={'CODGEO':str, 'LIBGEO':str, 'DEP':str})

arr_df = pd.read_excel('table-appartenance-geo-communes-18_V2.xls', sheet_name=1, skiprows=5, usecols=['CODGEO',
                                                                                                       'LIBGEO',
                                                                                                       'DEP'],
                       dtype={'CODGEO':str, 'LIBGEO':str, 'DEP':str}) 

city_df = com_df.append(arr_df, ignore_index=True)

# Delete leading O for 4 digits codes
city_df['CODGEO'].replace(to_replace='^0', value='', inplace=True, regex=True)

# Create a dictionnary for future mapping    
#city_dic = dict(zip(city_df['CODGEO'], city_df['LIBGEO']))
dep_dic = dict(zip(city_df['CODGEO'], city_df['DEP']))

In [3]:
# File with list of department "capitals"
chef_lieux_df = pd.read_excel('dico_cheflieux.xlsx', header=None, dtype={0:str, 1:str, 2:str, 3:str, 4:str})

chef_lieux_dic = dict(zip(chef_lieux_df[0], chef_lieux_df[1]))
north_south_dic = dict(zip(chef_lieux_df[0], chef_lieux_df[2]))
dpt_dic = dict(zip(chef_lieux_df[0], chef_lieux_df[4]))

In [4]:
# Function to get coordinates from cities
def geocoding_dpt(city): 
    geolocator = Nominatim(user_agent="mapviz", timeout=3)
    location = geolocator.geocode(city)
    
    return location.latitude, location.longitude

In [5]:
# Creation of a dictionnary to map coordinates with departments
lat_lon_dic = {k:geocoding_dpt(v) for (k,v) in chef_lieux_dic.items()}

In [34]:
# File too big (1GB) pandas is struggling 
# Chunk the file, one million rows per chunk (approximately 8 million rows total)
df_chunk = pd.read_csv('FD_MOBPRO_2016.csv', sep=';', usecols=['COMMUNE', 'DCLT', 'ILT'],
                chunksize=1000000, dtype={'COMMUNE':str, 'DCLT':str, 'ILT':str})

In [35]:
# Preprocessing the data, create a function to apply to each chunk
def chunk_preprocessing(chunk):
    # We only want to see migrations meaning reseidence area of 2016 different from place of work
    # Drop rows where COMMUNE equals DCLT
    chunk = chunk[chunk['COMMUNE'] != chunk['DCLT']]
    
    # Drop rows for people living abroad and we only want people moving to a different regions - Column ILT
    chunk = chunk[(chunk['ILT'] == '4')]
    
    # Delete leading O for 4 digits codes
    chunk['COMMUNE'].replace(to_replace='^0', value='', inplace=True, regex=True)
    chunk['DCLT'].replace(to_replace='^0', value='', inplace=True, regex=True)
    
    # Important to convert COMMUNE codes to actual understandable departments numbers
    chunk['dept_res'] = chunk['COMMUNE'].map(lambda x: dep_dic[x])
    
    # Important to convert DCLT codes to actual understandable departments numbers
    chunk['dept_work'] = chunk['DCLT'].map(lambda x: dep_dic[x])
    
    # From department get latitude and longitude of departments (the main city of each)
    chunk['lat_res'] = chunk['dept_res'].map(lambda x: lat_lon_dic[x][0])
    chunk['lon_res'] = chunk['dept_res'].map(lambda x: lat_lon_dic[x][1])
    chunk['lat_work'] = chunk['dept_work'].map(lambda x: lat_lon_dic[x][0])
    chunk['lon_work'] = chunk['dept_work'].map(lambda x: lat_lon_dic[x][1])
    
    # Drop unnecessary columns
    chunk = chunk.drop(axis=1, labels=['COMMUNE', 'DCLT', 'ILT'])
             
    return chunk


In [36]:
# Append each chunk to a list
chunk_list = [chunk_preprocessing(chunk) for chunk in df_chunk]
# Concatenate the list into a single DataFrame
df = pd.concat(chunk_list)

In [37]:
df.head()

,dept_res,dept_work,lat_res,lon_res,lat_work,lon_work
48,01,04,46.205119,5.225032,44.091814,6.235143
49,01,35,46.205119,5.225032,48.111339,-1.680020
72,01,71,46.205119,5.225032,46.303668,4.832227
73,01,71,46.205119,5.225032,46.303668,4.832227
1921,01,06,46.205119,5.225032,43.700936,7.268391


In [38]:
# We want to see migrations department to department
df = df[['dept_res', 'lat_res', 'lon_res', 'dept_work', 'lat_work', 'lon_work']]
network_df = df.copy()

In [39]:
network_df = network_df.groupby(['dept_res', 'lat_res', 'lon_res',
                                 'dept_work', 'lat_work', 'lon_work']).size().reset_index(name='flow_weight')
network_df

,dept_res,lat_res,lon_res,dept_work,lat_work,lon_work,flow_weight
0,01,46.205119,5.225032,04,44.091814,6.235143,5
1,01,46.205119,5.225032,05,44.561198,6.082002,1
2,01,46.205119,5.225032,06,43.700936,7.268391,14
3,01,46.205119,5.225032,08,49.773571,4.720694,1
4,01,46.205119,5.225032,09,42.963900,1.605381,1
...,...,...,...,...,...,...,...
7123,974,48.935773,2.358023,91,48.629610,2.443596,7
7124,974,48.935773,2.358023,92,48.892427,2.207127,16
7125,974,48.935773,2.358023,93,48.906387,2.445223,13
7126,974,48.935773,2.358023,94,48.777149,2.453073,5


In [40]:
# in order to get a clearer vizualisation, ouverseas departments are dropped 
network_df = network_df[(network_df['dept_work'] != '971') & (network_df['dept_work'] != '972') & 
                       (network_df['dept_work'] != '973') & (network_df['dept_work'] != '974') &
                       (network_df['dept_work'] != '975') & (network_df['dept_work'] != '976') &
                       (network_df['dept_work'] != '977') & (network_df['dept_work'] != '978') &
                       (network_df['dept_work'] != '986') & (network_df['dept_work'] != '987') &
                       (network_df['dept_work'] != '988')]
network_df = network_df[(network_df['dept_res'] != '971') & (network_df['dept_res'] != '972') & 
                       (network_df['dept_res'] != '973') & (network_df['dept_res'] != '974') &
                       (network_df['dept_res'] != '975') & (network_df['dept_res'] != '976') &
                       (network_df['dept_res'] != '977') & (network_df['dept_res'] != '978') &
                       (network_df['dept_res'] != '986') & (network_df['dept_res'] != '987') &
                       (network_df['dept_res'] != '988')]

# To much paths on the map, under 100 people per path it is not significant 
network_df = network_df[network_df['flow_weight'] >= 100]

network_df

,dept_res,lat_res,lon_res,dept_work,lat_work,lon_work,flow_weight
29,01,46.205119,5.225032,39,46.673902,5.558617,314
53,01,46.205119,5.225032,71,46.303668,4.832227,1949
79,02,49.564665,3.620686,08,49.773571,4.720694,117
113,02,49.564665,3.620686,51,48.956622,4.362885,1814
133,02,49.564665,3.620686,75,48.856697,2.351462,1278
...,...,...,...,...,...,...,...
6838,94,48.777149,2.453073,59,50.636565,3.063528,103
6839,94,48.777149,2.453073,60,49.430100,2.082336,164
6848,94,48.777149,2.453073,69,45.757814,4.832011,109
6891,95,49.050885,2.100807,27,49.026890,1.151016,125


In [45]:
# Séparation Nord Sud
geo_df = network_df.copy()

north_df = geo_df[(geo_df['lat_res'] > geo_df['lat_work'])]
south_df = geo_df[(geo_df['lat_res'] < geo_df['lat_work'])]

# Creating list of nodes with geo coordinates
cols = ['dept_res', 'lat_res', 'lon_res']
cols2 = ['dept_work', 'lat_work', 'lon_work']
department_ns = north_df[cols].values.tolist() + north_df[cols2].values.tolist()
department_sn = south_df[cols].values.tolist() + south_df[cols2].values.tolist()

import itertools
department_ns.sort()
department_sn.sort()

nodes_ns = list(x for x,_ in itertools.groupby(department_ns))
nodes_sn = list(x for x,_ in itertools.groupby(department_sn))

In [51]:
# Get 2 DF one for north-south movements and other for south-north ones
ants_ns = pd.DataFrame(nodes_ns)
ants_ns.columns = ['dpt', 'latitude', 'longitude']
ants_ns['name'] = ants_ns['dpt'].map(lambda x: dpt_dic[x])

ants_sn = pd.DataFrame(nodes_sn)
ants_sn.columns = ['dpt', 'latitude', 'longitude']
ants_sn['name'] = ants_sn['dpt'].map(lambda x: dpt_dic[x])

In [52]:
south_df.groupby('dept_work').sum().sort_values('flow_weight', ascending=False).head(10)

,lat_res,lon_res,lat_work,lon_work,flow_weight
dept_work,,,,,
75,1892.757684,116.545746,2003.124573,96.409926,15782
92,1113.187555,51.363451,1173.418255,52.971041,5719
78,235.056971,5.968705,244.017702,10.633443,5676
60,390.853902,17.496213,395.440798,16.658684,3948
84,87.448667,8.236802,87.898499,9.611802,3798
71,137.403080,14.444350,138.911005,14.496680,2993
95,452.796995,22.927756,490.508845,21.008067,2630
35,142.763663,-2.878045,144.334016,-5.040059,2453
77,143.996025,9.553811,145.619781,7.982451,2439


In [53]:
north_df.groupby('dept_work').sum().sort_values('flow_weight', ascending=False).head(10)

,lat_res,lon_res,lat_work,lon_work,flow_weight
dept_work,,,,,
75,544.658718,25.163943,537.423666,25.866078,14080
95,299.257008,14.933432,294.305307,12.604840,9739
93,397.240520,20.447333,391.251096,19.561785,5455
92,495.543017,26.254607,488.924273,22.071267,5267
78,248.825063,10.168067,244.017702,10.633443,4090
13,226.996405,20.312356,216.480871,26.849763,3651
77,198.587952,13.129435,194.159708,10.643268,2698
44,289.914053,-6.498142,283.311823,-9.324817,2637
69,339.694474,22.394109,320.304696,33.824080,2109


In [54]:
# Convert to file and export to local disk

#ants_ns.to_csv(r'/home/willy/Fr_migrations/cities_ns.csv', index = False, header=True)
#ants_sn.to_csv(r'/home/willy/Fr_migrations/cities_sn.csv', index = False, header=True)
#north_df.to_csv(r'/home/willy/Fr_migrations/moves_NS.csv', index = False, header=True)
#south_df.to_csv(r'/home/willy/Fr_migrations/moves_SN.csv', index = False, header=True)